In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.ao.quantization
import torch.ao.nn

torch.manual_seed(1)

In [2]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [3]:
class QuantNGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(QuantNGramLanguageModeler, self).__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.embeddings = torch.ao.nn.quantized.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, inputs):
        qinputs = self.quant(inputs)
        embeds = self.embeddings(qinputs.int()).view((1, -1))
        out = F.relu(self.linear1(embeds.float()))
        out = self.linear2(out)
        out = self.dequant(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [4]:
def training_loop(model):
    losses = []
    loss_function = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    for epoch in range(10):
        total_loss = 0
        for context, target in ngrams:

            # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
            # into integer indices and wrap them in tensors)
            context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.float)

            # Step 2. Recall that torch *accumulates* gradients. Before passing in a
            # new instance, you need to zero out the gradients from the old
            # instance
            model.zero_grad()

            # Step 3. Run the forward pass, getting log probabilities over next
            # words
            log_probs = model(context_idxs)

            # Step 4. Compute your loss function. (Again, Torch wants the target
            # word wrapped in a tensor)
            loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    print(losses)  # The loss decreased every iteration over the training data! 

In [5]:
model = QuantNGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
model.eval()
model.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')
model_prepared = torch.ao.quantization.prepare_qat(model.train())

/Users/Miguel/Desktop/MIT/spring2024/6.5610/65610_project/venv_embeddings/lib/python3.10/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [6]:
training_loop(model_prepared)

[4251.218174934387, 2979.6950171366334, 2231.616878427565, 1574.5523364543915, 1174.4323370456696, 753.2356539964676, 598.8750112201087, 507.3967487961054, 435.8005963098258, 384.04318079166114]


In [7]:
model_prepared.embeddings.weight()[word_to_ix["beauty"]]

tensor([ 96.,  43., 224.,  35.,   1.,   0.,   0.,   0., 111.,   4.],
       size=(10,), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=1.0, zero_point=0)

In [ ]:
model_prepared.eval()
quant_model = torch.ao.quantization.convert(model_prepared)

RuntimeError: Didn't find engine for operation quantized::linear_prepack NoQEngine